In [56]:
from config import *
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as sqlio
pd.set_option('display.max_columns', None)

In [182]:
#conn = pg.connect(f"host={aact_host} port={aact_port} dbname={aact_dbname} user={aact_user} password={aact_password}")
conn = pg.connect(host="localhost", port=5432, database="aact-fin")
null_signifiers = ["None", "none", "NONE", "NA", "N/A", "na", "n/a", "nan", "NaN", "Null", "null", "NULL", ""]

In [60]:
conn.close()

In [62]:
def run_query(conn, sql):
    cur = conn.cursor()
    dat = sqlio.read_sql_query(sql, conn)
    cur.close()
    return(dat)

In [63]:
sql = """select distinct 
    oa.nct_id,
    intv.name, intv.intervention_type,
    cnd.downcase_name,
    oa.outcome_id, oa.param_type, oa.param_value, oa.ci_lower_limit, oa.ci_upper_limit, oa.ci_percent, oa.p_value, oa.method
from 
    ctgov.outcome_analyses oa
    LEFT JOIN ctgov.interventions intv on intv.nct_id = oa.nct_id
    LEFT JOIN ctgov.conditions cnd on cnd.nct_id = oa.nct_id
"""
sql2 = """select 
        out.id, out.nct_id, out.title, out.time_frame, out.population, out.units_analyzed
from
    ctgov.outcomes out
"""


In [64]:
results = run_query(conn,sql)
outcomes = run_query(conn, sql2)
output = results.merge(outcomes, how="left", left_on="outcome_id", right_on="id")
output.to_csv("clintrials_results.csv")

In [65]:
####What are missing data fields?
#intervention
intervention = """select distinct intervention_model from ctgov.Designs """
intervention_results = run_query(conn, intervention)

In [51]:
intervention_results


,intervention_model
0,Parallel Assignment
1,None
2,Crossover Assignment
3,Sequential Assignment
4,Factorial Assignment
5,Single Group Assignment


In [10]:
intervention_results.dropna(inplace=True)
intervention_results = intervention_results[intervention_results['intervention_model'].str.len() != 0]
inverse_boolean_series = ~intervention_results['intervention_model'].isin(null_signifiers)
intervention_results = intervention_results[inverse_boolean_series]
intervention_results

,intervention_model
0,Parallel Assignment
2,Crossover Assignment
3,Sequential Assignment
4,Factorial Assignment
5,Single Group Assignment


In [66]:
#eligibilities
eligibility = """select distinct gender from ctgov.Eligibilities """
eligibility_results = run_query(conn, eligibility)

In [13]:
eligibility_results

,gender
0,None
1,Male
2,
3,Female
4,All


In [15]:
eligibility_results.dropna(inplace=True)
eligibility_results = eligibility_results[eligibility_results['gender'].str.len() != 0]
inverse_boolean_series = ~eligibility_results['gender'].isin(null_signifiers)
eligibility_results = eligibility_results[inverse_boolean_series]
eligibility_results

,gender
1,Male
3,Female
4,All


In [67]:
studies = """select distinct study_type from ctgov.Studies """
studies_results = run_query(conn, studies)

In [27]:
studies_results

,study_type
0,Observational [Patient Registry]
1,Interventional
2,Expanded Access
3,None
4,N/A
5,Observational


In [22]:
studies_results.dropna(inplace=True)
studies_results = studies_results[studies_results['study_type'].str.len() != 0]
inverse_boolean_series = ~studies_results['study_type'].isin(null_signifiers)
studies_results = studies_results[inverse_boolean_series]
studies_results

,study_type
0,Observational [Patient Registry]
1,Interventional
2,Expanded Access
5,Observational


In [68]:
##Number of studies
studies_no = """select distinct nct_id from ctgov.Studies """
studies_no_results = run_query(conn, studies_no)
studies_no_results.shape

(379094, 1)

In [179]:
def number_field(my_field, my_table): 
    sql_query = """select {ff} from ctgov.{tt} """.format(ff=my_field,tt=my_table)
    sql_query_results = run_query(conn, sql_query)
    sql_query_results.dropna(inplace=True)
    inverse_boolean_series = ~sql_query_results[my_field].isin(null_signifiers)
    sql_query_results = sql_query_results[inverse_boolean_series]
    return (sql_query_results)

In [180]:
def number_unique(my_field, my_table): 
    sql_query = """select {ff} from ctgov.{tt} """.format(ff=my_field,tt=my_table)
    sql_query_results = run_query(conn, sql_query)
    my_cols = sql_query_results.columns
    sql_query_results.dropna(inplace=True)
    inverse_boolean_series = ~sql_query_results[my_field].isin(null_signifiers)
    sql_query_results = sql_query_results[inverse_boolean_series]
    return (sql_query_results[my_field].nunique())

In [181]:
def number_study(my_field, my_table): 
    sql_query = """select nct_id, {ff} from ctgov.{tt} """.format(ff=my_field,tt=my_table)
    sql_query_results = run_query(conn, sql_query)
    sql_query_results.dropna(inplace=True)
    inverse_boolean_series = ~sql_query_results[my_field].isin(null_signifiers)
    sql_query_results = sql_query_results[inverse_boolean_series]
    return (sql_query_results['nct_id'].nunique())    

In [93]:
def average_study(my_field, my_table): 
    sql_query = """SELECT AVG(a.rcount) FROM 
    (select count({ff}) as rcount 
    FROM ctgov.{tt} b
    GROUP BY b.nct_id) a""".format(ff=my_field,tt=my_table)
    sql_query_results = run_query(conn, sql_query)
    return(sql_query_results)

In [85]:
##Number of Brief Summaries
brief_sum_no_results = number_field("description", "Brief_Summaries")
brief_sum_no_results

,description
0,\n This study was designed to evaluate th...
1,"\n This multicenter, observer-blind, cont..."
2,\n Bone metastases occur frequently durin...
3,"\n This is a Phase 2, randomized, open-la..."
4,\n Patients undergoing extracorporeal mem...
...,...
378253,\n This study will test the efficacy and ...
378254,\n RATIONALE: Internal radiation therapy ...
378255,\n This study is intended to provide acce...
378256,\n Etanercept is a novel anti-inflammator...


In [86]:
#Number of unique brief summaries
brief_uniq_no_results = number_unique("description", "Brief_Summaries")
brief_uniq_no_results

376213

In [87]:
#Number of studies with brief summary
brief_sum_no_studies = number_study("description", "Brief_Summaries")
brief_sum_no_studies

378258

In [47]:
#Avg number of Brief Summaries per study
brief_sum_no_avg_results = average_study("description", "Brief_Summaries")
brief_sum_no_avg_results

,avg
0,1.0


In [88]:
##Number of Condition Mesh terms 
cond_no_results = number_field("mesh_term", "Browse_Conditions")
cond_no_results

,mesh_term
0,Breast Neoplasms
1,Siderosis
2,Prostatic Neoplasms
3,Meningococcal Infections
4,Myofascial Pain Syndromes
...,...
638484,Hemolysis
638485,Atrophy
638486,Heart Diseases
638487,"Heart Defects, Congenital"


In [98]:
#Number of unique Condition Mesh terms
cond_uniq_no_results = number_unique("mesh_term", "Browse_Conditions")
cond_uniq_no_results

4045

In [91]:
#Number of studies with Condition Mesh term
cond_no_studies = number_study("mesh_term", "Browse_Conditions")
cond_no_studies

312326

In [94]:
#Avg number of Condition Mesh terms per study 
cond_no_avg_results = average_study("mesh_term", "Browse_Conditions")
cond_no_avg_results

,avg
0,2.044303


In [96]:
##Number of Intervention Mesh terms 
inter_no_results = number_field("mesh_term", "Browse_Interventions")
inter_no_results

,mesh_term
0,Pembrolizumab
1,Nivolumab
2,Durvalumab
3,Atezolizumab
4,Immunoglobulins
...,...
256785,Vaccines
256786,Calcium
256787,DMP 777
256788,Timolol


In [99]:
#Number of unique Intervention Mesh terms
inter_uniq_no_results = number_unique("mesh_term", "Browse_Interventions")
inter_uniq_no_results

3382

In [95]:
#Number of studies with Intervention Mesh term
inter_no_studies = number_study("mesh_term", "Browse_Interventions")
inter_no_studies

131327

In [97]:
#Avg number of Intervention Mesh terms per study 
inter_no_avg_results = average_study("mesh_term", "Browse_Interventions")
inter_no_avg_results

,avg
0,1.955348


In [108]:
##Number of Intervention Model terms 
inter_model_no_results = number_field("intervention_model", "Designs")
inter_model_no_results

,intervention_model
0,Parallel Assignment
1,Parallel Assignment
2,Single Group Assignment
3,Parallel Assignment
5,Single Group Assignment
...,...
379087,Single Group Assignment
379088,Single Group Assignment
379089,Parallel Assignment
379091,Single Group Assignment


In [103]:
##Number of unique terms 
inter_model_uniq_no_results = number_unique("intervention_model", "Designs")
inter_model_uniq_no_results

5

In [109]:
#Number of studies with term
inter_model_no_studies = number_study("intervention_model", "Designs")
inter_model_no_studies

288545

In [110]:
#Avg number of terms per study 
inter_model_no_avg_results = average_study("intervention_model", "Designs")
inter_model_no_avg_results

,avg
0,0.761144


In [111]:
##Number of terms 
inter_model_desc_no_results = number_field("intervention_model_description", "Designs")
inter_model_desc_no_results

,intervention_model_description
7,Randomized trial in which participants are ass...
8,Effects on outcomes when using mNavigator will...
13,Parallel-group
15,All subjects will collect 4 urine samples on D...
17,Trial with single arm
...,...
378965,Double Blinded Placebo Controlled Factorial
378967,"single-arm, open label"
379046,CREST-2 is two parallel multi-center randomize...
379088,Initiation of Study is defined as the day the ...


In [112]:
##Number of unique terms 
inter_model_desc_uniq_no_results = number_unique("intervention_model_description", "Designs")
inter_model_desc_uniq_no_results

33734

In [113]:
#Number of studies with term
inter_model_desc_no_studies = number_study("intervention_model_description", "Designs")
inter_model_desc_no_studies

36658

In [114]:
inter_model_desc_no_avg_results = average_study("intervention_model_description", "Designs")
inter_model_desc_no_avg_results

,avg
0,0.096699


In [118]:
##Number of terms 
prim_purpose_no_results = number_field("primary_purpose", "Designs")
prim_purpose_no_results["primary_purpose"]

0         Health Services Research
1                            Other
2                        Treatment
3                        Treatment
5                        Treatment
                    ...           
379087                   Treatment
379088                   Treatment
379089                   Treatment
379091                   Treatment
379092                   Treatment
Name: primary_purpose, Length: 287600, dtype: object

In [116]:
##Number of unique terms 
prim_purpose_uniq_no_results = number_unique("primary_purpose", "Designs")
prim_purpose_uniq_no_results

10

In [119]:
#Number of studies with term
prim_purpose_no_studies = number_study("primary_purpose", "Designs")
prim_purpose_no_studies

287600

In [120]:
#Avg number of terms per study 
prim_purpose_no_avg_results = average_study("primary_purpose", "Designs")
prim_purpose_no_avg_results

,avg
0,0.758651


In [124]:
##Number of terms 
allocation_no_results = number_field("allocation", "Designs")
allocation_no_results

,allocation
0,Randomized
1,Randomized
3,Randomized
6,Randomized
7,Randomized
...,...
379083,Randomized
379085,Randomized
379086,Randomized
379089,Randomized


In [122]:
##Number of unique terms 
allocation_uniq_no_results = number_unique("allocation", "Designs")
allocation_uniq_no_results

2

In [125]:
#Number of studies with term
allocation_no_studies = number_study("allocation", "Designs")
allocation_no_studies

224601

In [126]:
#Avg number of terms per study 
allocation_no_avg_results = average_study("allocation", "Designs")
allocation_no_avg_results

,avg
0,0.763141


In [129]:
##Number of terms 
det_des_no_results = number_field("description", "Detailed_Descriptions")
det_des_no_results

,description
0,\n This is a retrospective non-interventi...
1,\n Parkinson's Disease (PD) is a degenera...
2,\n Sudden cardiac death (SCD) is a major ...
3,\n Long title: Role of different investig...
4,\n The study will take place at UTI Dr. J...
...,...
252097,\n Sensorineural hearing loss accounts fo...
252098,\n Inpatient Use of Omegaven:\r\n\r\n ...
252099,\n OBJECTIVES:\r\n\r\n - Provide ...
252100,\n Hemodialysis patients with end stage r...


In [130]:
##Number of unique terms 
det_des_uniq_no_results = number_unique("description", "Detailed_Descriptions")
det_des_uniq_no_results

250444

In [131]:
#Number of studies with term
det_des_no_studies = number_study("description", "Detailed_Descriptions")
det_des_no_studies

252102

In [132]:
#Avg number of terms per study 
det_des_no_avg_results = average_study("description", "Detailed_Descriptions")
det_des_no_avg_results

,avg
0,1.0


In [134]:
##Number of terms 
interv_no_results = number_field("name", "Interventions")
interv_no_results

,name
0,Zimberelimab
1,Perianal Exam
2,high-resolution anoscopy (HRA) with biopsy
3,Standard of Care Surgical Procedure
4,VersaWrap Treatment
...,...
649226,Placebo
649227,Yttrium Y 90 glass microspheres
649228,"CP-675,206 (Tremelimumab)"
649229,Etanercept


In [135]:
##Number of unique terms 
interv_uniq_no_results = number_unique("name", "Interventions")
interv_uniq_no_results

328957

In [136]:
#Number of studies with term
interv_no_studies = number_study("name", "Interventions")
interv_no_studies

339579

In [137]:
#Avg number of terms per study 
interv_no_avg_results = average_study("name", "Interventions")
interv_no_avg_results

,avg
0,1.91187


In [138]:
##Number of terms 
interv_desc_no_results = number_field("description", "Interventions")
interv_desc_no_results


,description
0,Zimberelimab is a fully human immunoglobulin G...
1,Perianal Exam is part of anal cancer screening...
2,The HRA exam involves insertion of a plastic a...
3,Zone 2 Flexor Tendon surgical repair will be p...
4,is intended to manage and protect tendon injur...
...,...
649226,placebo
649227,"Between 0.2 to 0.5 Curies (200-500-mCi), eithe..."
649228,15 mg/kg IV every 3 months as long as required
649229,Hemodialysis patients will receive Etanercept ...


In [139]:
##Number of unique terms 
interv_desc_uniq_no_results = number_unique("description", "Interventions")
interv_desc_uniq_no_results

477127

In [140]:
#Number of studies with term
interv_desc_no_studies = number_study("description", "Interventions")
interv_desc_no_studies

292979

In [141]:
#Avg number of terms per study 
interv_desc_no_avg_results = average_study("description", "Interventions")
interv_desc_no_avg_results

,avg
0,1.642416


In [145]:
##Number of terms 
interv_type_no_results = number_field("intervention_type", "Interventions")
interv_type_no_results

,intervention_type
0,Drug
1,Procedure
2,Procedure
3,Procedure
4,Device
...,...
649226,Drug
649227,Radiation
649228,Drug
649229,Drug


In [143]:
##Number of unique terms 
interv_type_uniq_no_results = number_unique("intervention_type", "Interventions")
interv_type_uniq_no_results

11

In [146]:
#Number of studies with term
interv_type_no_studies = number_study("intervention_type", "Interventions")
interv_type_no_studies

339579

In [147]:
#Avg number of terms per study 
interv_type_no_avg_results = average_study("intervention_type", "Interventions")
interv_type_no_avg_results

,avg
0,1.91187


In [148]:
##Number of terms 
gender_no_results = number_field("gender", "Eligibilities")
gender_no_results

,gender
0,All
1,Female
2,All
3,All
4,All
...,...
379089,All
379090,All
379091,All
379092,All


In [149]:
##Number of unique terms 
gender_uniq_no_results = number_unique("gender", "Eligibilities")
gender_uniq_no_results

3

In [150]:
#Number of studies with term
gender_no_studies = number_study("gender", "Eligibilities")
gender_no_studies

378237

In [151]:
#Avg number of terms per study 
gender_no_avg_results = average_study("gender", "Eligibilities")
gender_no_avg_results

,avg
0,1.0


In [152]:
##Number of terms 
minage_no_results = number_field("minimum_age", "Eligibilities")
minage_no_results

,minimum_age
0,16 Years
1,18 Years
2,18 Years
3,18 Years
4,18 Years
...,...
379086,18 Years
379089,18 Years
379090,18 Years
379091,18 Years


In [153]:
##Number of unique terms 
minage_uniq_no_results = number_unique("minimum_age", "Eligibilities")
minage_uniq_no_results

246

In [154]:
#Number of studies with term
minage_no_studies = number_study("minimum_age", "Eligibilities")
minage_no_studies

347945

In [155]:
#Avg number of terms per study 
minage_no_avg_results = average_study("minimum_age", "Eligibilities")
minage_no_avg_results

,avg
0,1.0


In [156]:
##Number of terms 
maxage_no_results = number_field("maximum_age", "Eligibilities")
maxage_no_results

,maximum_age
3,78 Years
10,50 Years
11,85 Years
13,80 Years
14,100 Years
...,...
379080,40 Years
379085,69 Years
379088,5 Years
379091,130 Years


In [157]:
##Number of unique terms 
maxage_uniq_no_results = number_unique("maximum_age", "Eligibilities")
maxage_uniq_no_results

453

In [158]:
#Number of studies with term
maxage_no_studies = number_study("maximum_age", "Eligibilities")
maxage_no_studies

197530

In [159]:
#Avg number of terms per study 
maxage_no_avg_results = average_study("maximum_age", "Eligibilities")
maxage_no_avg_results

,avg
0,1.0


In [160]:
##Number of terms 
population_no_results = number_field("population", "Eligibilities")
population_no_results

,population
2,60 pts with severe AS but without any other co...
11,1000 Consecutive Patients who underwent Radial...
12,The base-population for the studies are:\r\n\r...
16,Patients who undergo colonoscopy for diarrhea ...
22,Healthy Newborns in the Studyregions: Braunsch...
...,...
379081,Workers and volunteers engaged or potentially ...
379082,Extra-pulmonary tuberculosis
379084,Participants will be selected from those indiv...
379090,Patients with metastatic cancer of the liver w...


In [162]:
##Number of unique terms 
population_uniq_no_results = number_unique("population", "Eligibilities")
population_uniq_no_results

74774

In [163]:
#Number of studies with term
population_no_studies = number_study("population", "Eligibilities")
population_no_studies

78155

In [164]:
#Avg number of terms per study 
population_no_avg_results = average_study("population", "Eligibilities")
population_no_avg_results

,avg
0,1.0


In [165]:
##Number of terms 
criteria_no_results = number_field("criteria", "Eligibilities")
criteria_no_results

,criteria
0,\n COMMON INCLUSION CRITERIA\r\n\r\n ...
1,\n Inclusion Criteria:\r\n\r\n ...
2,\n Inclusion Criteria:\r\n\r\n ...
3,\n Inclusion Criteria:\r\n\r\n ...
4,\n Inclusion Criteria:\r\n\r\n ...
...,...
379089,\n Inclusion Criteria:\r\n\r\n ...
379090,\n DISEASE CHARACTERISTICS:\r\n\r\n ...
379091,\n Inclusion Criteria:\r\n\r\n ...
379092,\n Inclusion Criteria:\r\n\r\n ...


In [166]:
##Number of unique terms 
criteria_uniq_no_results = number_unique("criteria", "Eligibilities")
criteria_uniq_no_results


374070

In [167]:
#Number of studies with term
criteria_no_studies = number_study("criteria", "Eligibilities")
criteria_no_studies

378167

In [168]:
#Avg number of terms per study 
criteria_no_avg_results = average_study("criteria", "Eligibilities")
criteria_no_avg_results

,avg
0,1.0


In [169]:
##Number of terms 
other_interv_no_results = number_field("name", "Intervention_Other_Names")
other_interv_no_results

,name
0,taxoter
1,Beckman Coulter Access SARS-CoV-2 IgG Antibody...
2,Mount Sinai COVID-19 ELISA IgG Antibody Test
3,allogeneic mesenchymal bone marrow cells
4,tozinameran
...,...
329242,Allogeneic adipose-derived adult mesenchymal s...
329243,ESN364
329244,(-)-Nicotine
329245,NIC


In [171]:
##Number of unique terms 
other_interv_uniq_no_results = number_unique("name", "Intervention_Other_Names")
other_interv_uniq_no_results

114363

In [172]:
#Number of studies with term
other_interv_no_studies = number_study("name", "Intervention_Other_Names")
other_interv_no_studies

110966

In [173]:
#Avg number of terms per study 
other_interv_no_avg_results = average_study("name", "Intervention_Other_Names")
other_interv_no_avg_results

,avg
0,2.967098


In [174]:
##Number of terms 
conditions_no_results = number_field("name", "Conditions")
conditions_no_results

,name
0,Acceptability of Health Care
1,Healthy Volunteers
2,Advanced and Metastatic Urothelial Cancer
3,Chronic Obstructive Pulmonary Disease (COPD)
4,SARS-CoV-2 Infection
...,...
643136,HIV
643137,Stroke
643138,Postneoadjuvant Therapy Stage III Gastric Canc...
643139,Keloid Scar


In [175]:
##Number of unique terms 
conditions_uniq_no_results = number_unique("name", "Conditions")
conditions_uniq_no_results

92343

In [176]:
#Number of studies with term
conditions_no_studies = number_study("name", "Conditions")
conditions_no_studies

378240

In [177]:
#Avg number of terms per study 
conditions_no_avg_results = average_study("name", "Conditions")
conditions_no_avg_results

,avg
0,1.700352


In [183]:
##Number of terms 
outcomeid_no_results = number_field("outcome_id", "Outcome_Analyses")
outcomeid_no_results

,outcome_id
0,6045647
1,6045648
2,6045649
3,6045650
4,6045651
...,...
209612,6492200
209613,6492200
209614,6492200
209615,6492201


In [184]:
##Number of unique terms 
outcomeid_uniq_no_results = number_unique("outcome_id", "Outcome_Analyses")
outcomeid_uniq_no_results

88721

In [185]:
#Number of studies with term
outcomeid_no_studies = number_study("outcome_id", "Outcome_Analyses")
outcomeid_no_studies

17234

In [186]:
#Avg number of terms per study 
outcomeid_no_avg_results = average_study("outcome_id", "Outcome_Analyses")
outcomeid_no_avg_results

,avg
0,12.162992


In [187]:
##Number of terms 
paramtype_no_results = number_field("param_type", "Outcome_Analyses")
paramtype_no_results

,param_type
16,Risk Ratio (RR)
17,Risk Ratio (RR)
18,Mean Difference (Final Values)
19,Mean Difference (Final Values)
20,Mean Difference (Final Values)
...,...
209612,LS Mean Difference
209613,LS Mean Difference
209614,LS Mean Difference
209615,LS Mean difference


In [188]:
##Number of unique terms 
paramtype_uniq_no_results = number_unique("param_type", "Outcome_Analyses")
paramtype_uniq_no_results

4583

In [189]:
#Number of studies with term
paramtype_no_studies = number_study("param_type", "Outcome_Analyses")
paramtype_no_studies

11290

In [190]:
#Avg number of terms per study 
paramtype_no_avg_results = average_study("param_type", "Outcome_Analyses")
paramtype_no_avg_results

,avg
0,12.162992


In [192]:
paramtype_no_results['param_type'].value_counts().to_csv("oa_param_type_value_counts.csv")

In [193]:
##Number of terms 
paramval_no_results = number_field("param_value", "Outcome_Analyses")
paramval_no_results

,param_value
16,1.319
17,1.715
18,5.070
19,4.380
20,11.300
...,...
209612,0.350
209613,-0.370
209614,-0.110
209615,-0.530


In [194]:
##Number of unique terms 
paramval_uniq_no_results = number_unique("param_value", "Outcome_Analyses")
paramval_uniq_no_results

21091

In [195]:
#Number of studies with term
paramval_no_studies = number_study("param_value", "Outcome_Analyses")
paramval_no_studies

11290

In [196]:
#Avg number of terms per study 
paramval_no_avg_results = average_study("param_value", "Outcome_Analyses")
paramval_no_avg_results

,avg
0,8.194731


In [200]:
##Number of terms 
distype_no_results = number_field("dispersion_type", "Outcome_Analyses")
distype_no_results


,dispersion_type
18,Standard Error of the Mean
19,Standard Error of the Mean
20,Standard Error of the Mean
23,Standard Error of the Mean
43,Standard Error of the Mean
...,...
209612,Standard Error of the Mean
209613,Standard Error of the Mean
209614,Standard Error of the Mean
209615,Standard Error of the Mean


In [198]:
##Number of unique terms 
distype_uniq_no_results = number_unique("dispersion_type", "Outcome_Analyses")
distype_uniq_no_results

2

In [201]:
#Number of studies with term
distype_no_studies = number_study("dispersion_type", "Outcome_Analyses")
distype_no_studies

2767

In [202]:
#Avg number of terms per study 
distype_no_avg_results = average_study("dispersion_type", "Outcome_Analyses")
distype_no_avg_results

,avg
0,12.162992


In [203]:
##Number of terms 
disval_no_results = number_field("dispersion_value", "Outcome_Analyses")
disval_no_results

,dispersion_value
18,0.793
19,0.821
20,2.672
23,0.861
43,65.730
...,...
209612,0.420
209613,0.420
209614,0.410
209615,0.440


In [204]:
##Number of unique terms 
disval_uniq_no_results = number_unique("dispersion_value", "Outcome_Analyses")
disval_uniq_no_results

8567

In [205]:
#Number of studies with term
disval_no_studies = number_study("dispersion_value", "Outcome_Analyses")
disval_no_studies


2767

In [206]:
#Avg number of terms per study 
disval_no_avg_results = average_study("dispersion_value", "Outcome_Analyses")
disval_no_avg_results

,avg
0,2.311942


In [208]:
##Number of terms 
lci_no_results = number_field("ci_lower_limit", "Outcome_Analyses")
lci_no_results

,ci_lower_limit
16,1.082
17,1.129
18,3.520
19,2.770
20,6.050
...,...
209612,-0.330
209613,-1.060
209614,-0.790
209615,-1.250


In [209]:
##Number of unique terms 
lci_uniq_no_results = number_unique("ci_lower_limit", "Outcome_Analyses")
lci_uniq_no_results

21361

In [210]:
#Number of studies with term
lci_no_studies = number_study("ci_lower_limit", "Outcome_Analyses")
lci_no_studies


10085

In [211]:
#Avg number of terms per study 
lci_no_avg_results = average_study("ci_lower_limit", "Outcome_Analyses")
lci_no_avg_results

,avg
0,7.689857


In [212]:
##Number of terms 
uci_no_results = number_field("ci_upper_limit", "Outcome_Analyses")
uci_no_results

,ci_upper_limit
16,1.607
17,2.605
18,6.630
19,5.990
20,16.540
...,...
209612,1.040
209613,0.320
209614,0.580
209615,0.190


In [213]:
##Number of unique terms 
uci_uniq_no_results = number_unique("ci_upper_limit", "Outcome_Analyses")
uci_uniq_no_results

22647

In [214]:
#Number of studies with term
uci_no_studies = number_study("ci_upper_limit", "Outcome_Analyses")
uci_no_studies

10081

In [215]:
#Avg number of terms per study 
uci_no_avg_results = average_study("ci_upper_limit", "Outcome_Analyses")
uci_no_avg_results

,avg
0,7.67245


In [216]:
##Number of terms 
cit_no_results = number_field("ci_percent", "Outcome_Analyses")
cit_no_results


,ci_percent
16,95.0
17,95.0
18,95.0
19,95.0
20,95.0
...,...
209612,90.0
209613,90.0
209614,90.0
209615,90.0


In [217]:
##Number of unique terms 
cit_uniq_no_results = number_unique("ci_percent", "Outcome_Analyses")
cit_uniq_no_results

140

In [218]:
#Number of studies with term
cit_no_studies = number_study("ci_percent", "Outcome_Analyses")
cit_no_studies

10486

In [219]:
#Avg number of terms per study 
cit_no_avg_results = average_study("ci_percent", "Outcome_Analyses")
cit_no_avg_results

,avg
0,7.838691


In [220]:
##Number of terms 
pval_no_results = number_field("p_value", "Outcome_Analyses")
pval_no_results

,p_value
0,0.010
1,0.030
2,0.005
3,0.560
4,0.240
...,...
209612,0.802
209613,0.189
209614,0.399
209615,0.111


In [221]:
##Number of unique terms 
pval_uniq_no_results = number_unique("p_value", "Outcome_Analyses")
pval_uniq_no_results

10974

In [222]:
#Number of studies with term
pval_no_studies = number_study("p_value", "Outcome_Analyses")
pval_no_studies

14682

In [223]:
#Avg number of terms per study 
pval_no_avg_results = average_study("p_value", "Outcome_Analyses")
pval_no_avg_results

,avg
0,9.924742


In [224]:
##Number of terms 
method_no_results = number_field("method", "Outcome_Analyses")
method_no_results

,method
0,Chi-squared
1,Wilcoxon (Mann-Whitney)
2,Wilcoxon (Mann-Whitney)
3,Wilcoxon (Mann-Whitney)
4,Wilcoxon (Mann-Whitney)
...,...
209612,Repeated Measures Model
209613,Repeated Measures Model
209614,Repeated Measures Model
209615,Repeated Measures Model


In [225]:
##Number of unique terms 
method_uniq_no_results = number_unique("method", "Outcome_Analyses")
method_uniq_no_results

2875

In [226]:
#Number of studies with term
method_no_studies = number_study("method", "Outcome_Analyses")
method_no_studies

14682

In [227]:
#Avg number of terms per study 
method_no_avg_results = average_study("method", "Outcome_Analyses")
method_no_avg_results

,avg
0,12.162992


In [229]:
method_no_results['method'].value_counts().to_csv("oa_method_value_counts.csv")

In [230]:
##Number of terms 
method_desc_no_results = number_field("method_description", "Outcome_Analyses")
method_desc_no_results

,method_description
16,Cochran-Mantel-Haenszel test stratified by ana...
17,Cochran-Mantel-Haenszel test stratified by ana...
18,"ANCOVA model includes treatment, Baseline infl..."
19,"ANCOVA model includes treatment, Baseline infl..."
20,"ANCOVA model includes treatment, Baseline infl..."
...,...
209612,P-value was based on repeated measures model f...
209613,P-value was based on repeated measures model f...
209614,P-value was based on repeated measures model f...
209615,P-value was based on repeated measures model f...


In [231]:
##Number of unique terms 
method_desc_uniq_no_results = number_unique("method_description", "Outcome_Analyses")
method_desc_uniq_no_results

9262

In [232]:
#Number of studies with term
method_desc_no_studies = number_study("method_description", "Outcome_Analyses")
method_desc_no_studies

4420

In [234]:
#Avg number of terms per study 
method_desc_no_avg_results = average_study("method_description", "Outcome_Analyses")
method_desc_no_avg_results

,avg
0,12.162992


In [235]:
##Number of terms 
estimate_desc_no_results = number_field("estimate_description", "Outcome_Analyses")
estimate_desc_no_results

,estimate_description
54,The p value in Comparator group at Day 5 over ...
55,The p value in RiaGev group at Day 5 over Day ...
56,The p value between RiaGev and Comparator grou...
57,The p value between RiaGev and Comparator grou...
58,The p value in Comparator group at Day 5 over ...
...,...
209411,Posterior Probability (PP) of true mean differ...
209420,Estimated ratio (%) of exponentiated mean diff...
209421,Estimated ratio (%) of exponentiated mean diff...
209467,Estimated value was calculated as the percenta...


In [236]:
##Number of unique terms 
estimate_desc_uniq_no_results = number_unique("estimate_description","Outcome_Analyses")
estimate_desc_uniq_no_results

12873

In [237]:
#Number of studies with term
estimate_desc_no_studies = number_study("estimate_description", "Outcome_Analyses")
estimate_desc_no_studies

4333

In [238]:
#Avg number of terms per study 
estimate_desc_no_avg_results = average_study("estimate_description", "Outcome_Analyses")
estimate_desc_no_avg_results

,avg
0,12.162992


In [239]:
##Number of terms 
group_desc_no_results = number_field("groups_description", "Outcome_Analyses")
group_desc_no_results

,groups_description
16,Statistical Analysis For Week 8
17,Statistical Analysis For Week 4
18,Statistical Analysis For Week 8
19,Statistical Analysis For Week 4
35,Comparative analysis of time to clinical statu...
...,...
209612,Week 2: LS Mean Difference was estimated from ...
209613,Week 1: LS Mean Difference was estimated from ...
209614,Week 1: LS Mean Difference was estimated from ...
209615,LS mean difference was estimated from the repe...


In [240]:
##Number of unique terms 
group_desc_uniq_no_results = number_unique("groups_description","Outcome_Analyses")
group_desc_uniq_no_results


56217

In [241]:
#Number of studies with term
group_desc_no_studies = number_study("groups_description", "Outcome_Analyses")
group_desc_no_studies

9291

In [242]:
#Avg number of terms per study 
group_desc_no_avg_results = average_study("groups_description", "Outcome_Analyses")
group_desc_no_avg_results

,avg
0,12.162992


In [246]:
##Number of terms 
outcome_typ_no_results = number_field("outcome_type", "Outcomes")
outcome_typ_no_results

array(['Secondary', 'Primary', 'Post-Hoc', 'Other Pre-specified'],
      dtype=object)

In [247]:
##Number of unique terms 
outcome_typ_uniq_no_results = number_unique("outcome_type", "Outcomes")
outcome_typ_uniq_no_results

4

In [248]:
#Number of studies with term
outcome_typ_no_studies = number_study("outcome_type", "Outcomes")
outcome_typ_no_studies

49275

In [249]:
#Avg number of terms per study 
outcome_typ_no_avg_results = average_study("outcome_type", "Outcomes")
outcome_typ_no_avg_results

,avg
0,7.816844


In [250]:
##Number of terms 
title_no_results = number_field("title", "Outcomes")
title_no_results

,title
0,Treatment-Related Adverse Events as Assessed b...
1,Rate of COVID-19 Polymerase Chain Reaction (PC...
2,Changes in Serum D-dimer Levels
3,Changes in Serum Ferritin Levels
4,Changes in the Ratio of Polymorphonuclear Leuk...
...,...
385170,Efficacy Endpoints: Tumor Status: Alive With D...
385171,Safety Endpoints: Hypersensitivity Reactions t...
385172,Safety Endpoints: Serious Adverse Events to Tr...
385173,C-reactive Protein


In [251]:
##Number of unique terms 
title_uniq_no_results = number_unique("title", "Outcomes")
title_uniq_no_results


316422

In [252]:
#Number of studies with term
title_no_studies = number_study("title", "Outcomes")
title_no_studies

49275

In [253]:
#Avg number of terms per study 
title_no_avg_results = average_study("title", "Outcomes")
title_no_avg_results

,avg
0,7.816844


In [254]:
##Number of terms 
desc_no_results = number_field("description", "Outcomes")
desc_no_results

,description
0,Adverse effects of ivermectin and drugs other ...
1,"At the end of the follow-up period (10th day),..."
2,"In both groups, after the treatment period was..."
3,"In both groups, after the treatment period was..."
4,"In both groups, after the treatment period was..."
...,...
385166,Participants assessed their average daily pain...
385167,Participants assessed their average daily pain...
385168,Time from the first dose of study drug to the ...
385173,A reduced C-reactive protein (CRP) concentrati...


In [255]:
##Number of unique terms 
desc_uniq_no_results = number_unique("description", "Outcomes")
desc_uniq_no_results

287598

In [256]:
#Number of studies with term
desc_no_studies = number_study("description", "Outcomes")
desc_no_studies

47484

In [257]:
#Avg number of terms per study 
desc_no_avg_results = average_study("description", "Outcomes")
desc_no_avg_results

,avg
0,7.816844


In [258]:
##Number of terms 
tf_no_results = number_field("time_frame", "Outcomes")
tf_no_results


,time_frame
0,From the 6th day of study to the 10th day of s...
1,At the end of 10th day
2,From 6th to the end of 10th day
3,From 6th to the end of 10th day
4,From 6th to the end of 10th day
...,...
385170,"Following the first dose, until at least 90 da..."
385171,"Following the first dose, until at least 90 da..."
385172,"Following the first dose, until at least 90 da..."
385173,52 weeks


In [259]:
##Number of unique terms 
tf_uniq_no_results = number_unique("time_frame", "Outcomes")
tf_uniq_no_results


110331

In [260]:
#Number of studies with term
tf_no_studies = number_study("time_frame", "Outcomes")
tf_no_studies

49269

In [261]:
#Avg number of terms per study 
tf_no_avg_results = average_study("time_frame", "Outcomes")
tf_no_avg_results

,avg
0,7.816844


In [262]:
##Number of terms 
pop_no_results = number_field("population", "Outcomes")
pop_no_results

,population
1,PCR test was applied to 8 patients in the cont...
13,We aimed to investigate the effectiveness/safe...
29,The Safety (SAF) population included all rando...
30,The ITT population included all randomized par...
31,The ITT population included all randomized par...
...,...
385170,Treated patients with response assessed
385171,37 Participants received study treatment
385172,37 Participants received study treatment
385173,"Malnutrition, inflammation, and atherosclerosi..."


In [263]:
##Number of unique terms 
pop_uniq_no_results = number_unique("population", "Outcomes")
pop_uniq_no_results

116412

In [264]:
#Number of studies with term
pop_no_studies = number_study("population", "Outcomes")
pop_no_studies

37694

In [265]:
#Avg number of terms per study 
pop_no_avg_results = average_study("population", "Outcomes")
pop_no_avg_results

,avg
0,7.816844


In [266]:
##Number of terms 
units_no_results = number_field("units", "Outcomes")
units_no_results

,units
0,Participants
1,Participants
2,mg/L
3,mg/dL
4,Ratio
...,...
385170,Participants
385171,Participants
385172,Participants
385173,mg/L


In [267]:
##Number of unique terms 
units_uniq_no_results = number_unique("units", "Outcomes")
units_uniq_no_results

29931

In [268]:
#Number of studies with term
units_no_studies = number_study("units", "Outcomes")
units_no_studies


47610

In [269]:
#Avg number of terms per study 
units_no_avg_results = average_study("units", "Outcomes")
units_no_avg_results

,avg
0,7.816844


In [270]:
##Number of terms 
unitsa_no_results = number_field("units_analyzed", "Outcomes")
unitsa_no_results

,units_analyzed
100,Test site
101,Test site
103,Test site
104,Test site
105,Test site
...,...
385117,Limbs
385118,Limbs
385119,Limbs
385120,Limbs


In [271]:
##Number of unique terms 
unitsa_uniq_no_results = number_unique("units_analyzed", "Outcomes")
unitsa_uniq_no_results

1052

In [272]:
#Number of studies with term
unitsa_no_studies = number_study("units_analyzed", "Outcomes")
unitsa_no_studies


1951

In [273]:
#Avg number of terms per study 
unitsa_no_avg_results = average_study("units_analyzed", "Outcomes")
unitsa_no_avg_results

,avg
0,7.816844


In [276]:
##Number of terms 
distyp_no_results = number_field("dispersion_type", "Outcomes")
distyp_no_results

,dispersion_type
2,Standard Deviation
3,Standard Deviation
4,Standard Deviation
5,Standard Deviation
6,Standard Deviation
...,...
385166,Standard Deviation
385167,Standard Deviation
385168,95% Confidence Interval
385173,Standard Deviation


In [277]:
##Number of unique terms 
distyp_uniq_no_results = number_unique("dispersion_type", "Outcomes")
distyp_uniq_no_results


31

In [278]:
#Number of studies with term
distyp_no_studies = number_study("dispersion_type", "Outcomes")
distyp_no_studies

38608

In [279]:
#Avg number of terms per study 
distyp_no_avg_results = average_study("dispersion_type", "Outcomes")
distyp_no_avg_results

,avg
0,7.816844


In [283]:
##Number of terms 
partyp_no_results = number_field("param_type", "Outcomes")
partyp_no_results


,param_type
0,Count of Participants
1,Count of Participants
2,Mean
3,Mean
4,Mean
...,...
385170,Count of Participants
385171,Count of Participants
385172,Count of Participants
385173,Mean


In [281]:
##Number of unique terms 
partyp_uniq_no_results = number_unique("param_type", "Outcomes")
partyp_uniq_no_results

9

In [284]:
#Number of studies with term
partyp_no_studies = number_study("param_type", "Outcomes")
partyp_no_studies

47612

In [285]:
#Avg number of terms per study 
partyp_no_avg_results = average_study("param_type", "Outcomes")
partyp_no_avg_results

,avg
0,7.816844


In [286]:
##Number of terms 
outunits_no_results = number_field("units", "Outcome_Counts")
outunits_no_results

,units
0,Participants
1,Participants
2,Participants
3,Participants
4,Participants
...,...
921939,Participants
921940,Participants
921941,Participants
921942,Participants


In [287]:
##Number of unique terms 
outunits_uniq_no_results = number_unique("units", "Outcome_Counts")
outunits_uniq_no_results

1453

In [292]:
#Number of studies with term
outunits_no_studies = number_study("units", "Outcome_Counts")
outunits_no_studies


49275

In [293]:
#Avg number of terms per study 
outunits_no_avg_results = average_study("units", "Outcome_Counts")
outunits_no_avg_results

,avg
0,18.710178


In [288]:
##Number of terms 
coutunits_no_results = number_field("count", "Outcome_Counts")
coutunits_no_results

,count
0,30.0
1,30.0
2,16.0
3,8.0
4,30.0
...,...
921939,37.0
921940,5.0
921941,5.0
921942,5.0


In [289]:
##Number of unique terms 
coutunits_uniq_no_results = number_unique("count", "Outcome_Counts")
coutunits_uniq_no_results

4934

In [290]:
#Number of studies with term
coutunits_no_studies = number_study("count", "Outcome_Counts")
coutunits_no_studies

49275

In [291]:
#Avg number of terms per study 
coutunits_no_avg_results = average_study("count", "Outcome_Counts")
coutunits_no_avg_results

,avg
0,18.709691


In [294]:
##Number of terms 
pmid_no_results = number_field("pmid", "Study_References")
pmid_no_results

,pmid
0,8915864
2,8001003
3,7095322
4,10995154
5,29500159
...,...
1166853,18349311
1166854,20497755
1166855,8195317
1166856,7540117


In [295]:
##Number of unique terms 
pmid_uniq_no_results = number_unique("pmid", "Study_References")
pmid_uniq_no_results


462776

In [296]:
#Number of studies with term
pmid_no_studies = number_study("pmid", "Study_References")
pmid_no_studies

127745

In [297]:
#Avg number of terms per study 
pmid_no_avg_results = average_study("pmid", "Study_References")
pmid_no_avg_results

,avg
0,8.523369


In [301]:
##Number of terms 
reftyp_no_results = number_field("reference_type", "Study_References")
reftyp_no_results

,reference_type
0,reference
1,reference
2,reference
3,results_reference
4,reference
...,...
1166853,results_reference
1166854,results_reference
1166855,reference
1166856,reference


In [299]:
##Number of unique terms 
reftyp_uniq_no_results = number_unique("reference_type", "Study_References")
reftyp_uniq_no_results

5

In [302]:
#Number of studies with term
reftyp_no_studies = number_study("reference_type", "Study_References")
reftyp_no_studies


129446

In [303]:
#Avg number of terms per study 
reftyp_no_avg_results = average_study("reference_type", "Study_References")
reftyp_no_avg_results

,avg
0,9.014245


In [304]:
##Number of terms 
cit_no_results = number_field("citation", "Study_References")
cit_no_results


,citation
0,"Cypher TJ, Grossman JP. Biological principles ..."
1,"Beck, A. T., Steer, R. A., & Brown, G. K. (199..."
2,"Lynch CF, Cohen MB. Urinary system. Cancer. 19..."
3,"Schow RL, Nerbonne MA. Communication screening..."
4,Cook TM. A grading system for direct laryngosc...
...,...
1166853,"Sato KT, Lewandowski RJ, Mulcahy MF, Atassi B,..."
1166854,"Don BR, Kim K, Li J, Dwyer T, Alexander F, Kay..."
1166855,Puck JM. Molecular and genetic basis of X-link...
1166856,"Fisher GH, Rosenberg FJ, Straus SE, Dale JK, M..."


In [305]:
##Number of unique terms 
cit_uniq_no_results = number_unique("citation", "Study_References")
cit_uniq_no_results


494002

In [306]:
#Number of studies with term
cit_no_studies = number_study("citation", "Study_References")
cit_no_studies

129446

In [307]:
#Avg number of terms per study 
cit_no_avg_results = average_study("citation", "Study_References")
cit_no_avg_results

,avg
0,9.014245


In [308]:
##Number of terms 
brief_tit_no_results = number_field("brief_title", "Studies")
brief_tit_no_results

,brief_title
0,Thrombosomes for Expanded Access Use
1,Grapiprant (ARY-007) and Pembrolizumab in Pati...
2,A Study of the C3 Complement Inhibitor AMY-101...
3,Predicting Diabetic Retinopathy From Risk Fact...
4,"A Study to Determine the Efficacy, Safety, Pha..."
...,...
379089,RN624 For Pain Of Post-Herpetic Neuralgia
379090,Radiolabeled Glass Beads in Treating Patients ...
379091,A Rollover Study for Patients Who Received Tre...
379092,Safety and Efficacy Study of the Effect of Eta...


In [309]:
##Number of unique terms 
brief_tit_uniq_no_results = number_unique("brief_title", "Studies")
brief_tit_uniq_no_results

377013

In [310]:
#Number of studies with term
brief_tit_no_studies = number_study("brief_title", "Studies")
brief_tit_no_studies

379094

In [311]:

#Avg number of terms per study 
brief_tit_no_avg_results = average_study("brief_title", "Studies")
brief_tit_no_avg_results

,avg
0,1.0


In [315]:
##Number of terms 
study_typ_no_results = number_field("study_type", "Studies")
study_typ_no_results

,study_type
0,Expanded Access
1,Interventional
2,Interventional
3,Observational
4,Interventional
...,...
379089,Interventional
379090,Observational
379091,Interventional
379092,Interventional


In [313]:
##Number of unique terms 
study_typ_no_results = number_unique("study_type", "Studies")
study_typ_no_results

4

In [316]:
#Number of studies with term
study_typ_no_results = number_study("study_type", "Studies")
study_typ_no_results

378259

In [317]:
#Avg number of terms per study 
study_typ_no_results = average_study("study_type", "Studies")
study_typ_no_results

,avg
0,1.0


In [318]:
##Number of terms 
bp_no_results = number_field("baseline_population", "Studies")
bp_no_results

,baseline_population
15,3 patients were excluded due to history of sto...
56,Baseline analysis population included all part...
516,Participants of both arms are included here
716,All enrolled participants
1159,Analysis was performed on all randomized parti...
...,...
379085,Baseline characteristics data was not collecte...
379088,Infants were included based on the eligible cr...
379089,Safety analysis population included all partic...
379091,One participant received no study medication a...


In [319]:
##Number of unique terms 
bp_no_results = number_unique("baseline_population", "Studies")
bp_no_results

12458

In [320]:
#Number of studies with term
bp_no_results = number_study("baseline_population", "Studies")
bp_no_results

14855

In [321]:
#Avg number of terms per study 
bp_no_results = average_study("baseline_population", "Studies")
bp_no_results

,avg
0,1.0


In [322]:
##Number of terms 
oftit_no_results = number_field("official_title", "Studies")
oftit_no_results

,official_title
0,Thrombosomes for Expanded Access Use
1,"Open Label, Single Arm, Phase 1b/2 Study to Ev..."
2,A Phase 2a Clinical Trial to Assess the Safety...
3,Predicting Diabetic Retinopathy From Risk Fact...
4,"A Phase 2, Open-label, Multicenter Study to De..."
...,...
379089,"A PHASE II RANDOMIZED, DOUBLE-BLIND, PLACEBO-C..."
379090,A Humanitarian Device Exemption Compassionate ...
379091,A Rollover Protocol For Patients Who Received ...
379092,The Effect of Etanercept in Suppression of the...


In [323]:
##Number of unique terms 
oftit_no_results = number_unique("official_title", "Studies")
oftit_no_results

366058

In [324]:
#Number of studies with term
oftit_no_results = number_study("official_title", "Studies")
oftit_no_results


369010

In [325]:
#Avg number of terms per study 
oftit_no_results = average_study("official_title", "Studies")
oftit_no_results

,avg
0,0.9734


In [328]:
##Number of terms 
ovstat_no_results = number_field("overall_status", "Studies")
ovstat_no_results

,overall_status
0,Available
1,Terminated
2,Completed
3,Recruiting
4,"Active, not recruiting"
...,...
379089,Completed
379090,"Active, not recruiting"
379091,"Active, not recruiting"
379092,Terminated


In [329]:
##Number of unique terms 
ovstat_no_results = number_unique("overall_status", "Studies")
ovstat_no_results

14

In [330]:
#Number of studies with term
ovstat_no_results = number_study("overall_status", "Studies")
ovstat_no_results


379094

In [331]:
#Avg number of terms per study 
ovstat_no_results = average_study("overall_status", "Studies")
ovstat_no_results

,avg
0,1.0


In [335]:
##Number of terms 
phase_no_results = number_field("phase", "Studies")
phase_no_results

,phase
1,Phase 1/Phase 2
2,Phase 1/Phase 2
4,Phase 2
5,Phase 4
8,Phase 4
...,...
379080,Phase 3
379087,Phase 2
379089,Phase 2
379091,Phase 2


In [333]:
##Number of unique terms 
phase_no_results = number_unique("phase", "Studies")
phase_no_results

7

In [336]:
#Number of studies with term
phase_no_results = number_study("phase", "Studies")
phase_no_results

166316

In [337]:
#Avg number of terms per study 
phase_no_results = average_study("phase", "Studies")
phase_no_results

,avg
0,0.778704


In [338]:
##Number of terms 
num_arms_no_results = number_field("number_of_arms", "Studies")
num_arms_no_results

,number_of_arms
1,1.0
2,2.0
4,1.0
5,2.0
6,2.0
...,...
379087,1.0
379088,1.0
379089,3.0
379091,1.0


In [339]:
##Number of unique terms 
num_arms_no_results = number_unique("number_of_arms", "Studies")
num_arms_no_results

37

In [340]:
#Number of studies with term
num_arms_no_results = number_study("number_of_arms", "Studies")
num_arms_no_results

271276

In [341]:
##Number of terms 
num_groups_no_results = number_field("number_of_groups", "Studies")
num_groups_no_results

,number_of_groups
3,1.0
18,1.0
21,3.0
26,1.0
27,2.0
...,...
379079,1.0
379081,1.0
379082,2.0
379090,1.0


In [342]:
##Number of unique terms 
num_groups_no_results = number_unique("number_of_groups", "Studies")
num_groups_no_results

26

In [343]:
#Number of studies with term
num_groups_no_results = number_study("number_of_groups", "Studies")
num_groups_no_results

52494

In [344]:
##Number of terms 
limcav_no_results = number_field("limitations_and_caveats", "Studies")
limcav_no_results

,limitations_and_caveats
15,Small number of study participants. Perhaps be...
1156,Despite promising efficacy and a good tolerabi...
1179,The Study was closed by the Sponsor as it was ...
1345,"Open-label design, no assessment by VFA or X-r..."
2399,Early termination secondary to slow recruitment.
...,...
378909,An early termination of the study led to a sma...
378974,The study was prematurely closed due to unsati...
378999,There are no overall Limitations and Caveats
379088,The study protocol was terminated early follow...


In [345]:
##Number of unique terms 
limcav_no_results = number_unique("limitations_and_caveats", "Studies")
limcav_no_results

9937

In [346]:
#Number of studies with term
limcav_no_results = number_study("limitations_and_caveats", "Studies")
limcav_no_results

10496

In [347]:
##Number of terms 
enrol_no_results = number_field("enrollment", "Studies")
enrol_no_results

,enrollment
1,18.0
2,39.0
3,300.0
4,20.0
5,30.0
...,...
379089,99.0
379090,2000.0
379091,37.0
379092,10.0


In [348]:
##Number of unique terms 
enrol_no_results = number_unique("enrollment", "Studies")
enrol_no_results

6929

In [349]:
#Number of studies with term
enrol_no_results = number_study("enrollment", "Studies")
enrol_no_results

372345

In [356]:
##Number of terms 
restyp_no_results = number_field("result_type", "Result_Groups")
restyp_no_results

,result_type
0,Baseline
1,Baseline
2,Baseline
3,Participant Flow
4,Participant Flow
...,...
1281213,Outcome
1281214,Outcome
1281215,Outcome
1281216,Reported Event


In [354]:
##Number of unique terms 
restyp_no_results = number_unique("result_type", "Result_Groups")
restyp_no_results

4

In [357]:
#Number of studies with term
restyp_no_results = number_study("result_type", "Result_Groups")
restyp_no_results

49276

In [358]:
#Avg number of terms per study 
restyp_no_results = average_study("result_type", "Result_Groups")
restyp_no_results

,avg
0,26.000852


In [359]:
##Number of terms 
restit_no_results = number_field("title", "Result_Groups")
restit_no_results

,title
0,Total
1,Study Group
2,Control Group
3,Study Group
4,Control Group
...,...
1281213,Etanercept
1281214,Placebo
1281215,Etanercept
1281216,Placebo


In [360]:
##Number of unique terms 
restit_no_results = number_unique("title", "Result_Groups")
restit_no_results

141329

In [361]:
#Number of studies with term
restit_no_results = number_study("title", "Result_Groups")
restit_no_results

49276

In [362]:
#Avg number of terms per study 
restit_no_results = average_study("title", "Result_Groups")
restit_no_results

,avg
0,26.000852


In [363]:
##Number of terms 
resdesc_no_results = number_field("description", "Result_Groups")
resdesc_no_results

,description
0,Total of all reporting groups
1,"In addition to HFA treatment, ivermectin 200 m..."
2,Patients who were hospitalised with a pre-diag...
3,"In addition to HFA treatment, ivermectin 200 m..."
4,Patients who were hospitalised with a pre-diag...
...,...
1281213,Active comparator: Etanercept 25 mg injection ...
1281214,No Drug: Saline injection twice a week\r\nHemo...
1281215,Active comparator: Etanercept 25 mg injection ...
1281216,Saline injection twice a week\r\nPlacebo: Hemo...


In [364]:
##Number of unique terms 
resdesc_no_results = number_unique("description", "Result_Groups")
resdesc_no_results


207982

In [365]:
#Number of studies with term
resdesc_no_results = number_study("description", "Result_Groups")
resdesc_no_results

49154

In [366]:
#Avg number of terms per study 
resdesc_no_results = average_study("description", "Result_Groups")
resdesc_no_results

,avg
0,26.000852


In [371]:
##Number of terms 
grptyp_no_results = number_field("group_type", "Design_Groups")
grptyp_no_results

,group_type
0,Experimental
1,Experimental
2,Experimental
3,Experimental
4,No Intervention
...,...
675091,Active Comparator
675092,Placebo Comparator
675094,Experimental
675095,Active Comparator


In [372]:
##Number of unique terms 
grptyp_no_results = number_unique("group_type", "Design_Groups")
grptyp_no_results

6

In [373]:
#Number of studies with term
grptyp_no_results = number_study("group_type", "Design_Groups")
grptyp_no_results

271276

In [374]:
#Avg number of terms per study 
grptyp_no_results = average_study("group_type", "Design_Groups")
grptyp_no_results

,avg
0,1.789091


In [375]:
##Number of terms 
grptit_no_results = number_field("title", "Design_Groups")
grptit_no_results


,title
0,Study Group
1,LithoVue ureteroscope system
2,WatchPAT 200 and In-Lab Study
3,"Patients, Neck-specific exercises"
4,Healthy controls
...,...
675094,1
675095,Etanercept 25 mg
675096,Saline
675097,Blood relatives


In [376]:
##Number of unique terms 
grptit_no_results = number_unique("title", "Design_Groups")
grptit_no_results

414092

In [378]:
#Number of studies with term
grptit_no_results = number_study("title", "Design_Groups")
grptit_no_results

323770

In [379]:
#Avg number of terms per study 
grptit_no_results = average_study("title", "Design_Groups")
grptit_no_results

,avg
0,2.085119


In [380]:
##Number of terms 
grpdesc_no_results = number_field("description", "Design_Groups")
grpdesc_no_results

,description
0,Patients will receive both the clinically indi...
1,The LithoVue System is intended to be used to ...
2,Patients who are referred to the sleep center ...
3,Patients before and after neck-specific exerci...
4,"Healthy controls, no treatment"
...,...
675094,"Drug: CP-675,206 (Tremelimumab)"
675095,Etanercept 25 mg injection twice a week
675096,Saline injection twice a week
675097,blood related family members of proband


In [381]:
##Number of unique terms 
grpdesc_no_results = number_unique("description", "Design_Groups")
grpdesc_no_results

548607

In [382]:
#Number of studies with term
grpdesc_no_results = number_study("description", "Design_Groups")
grpdesc_no_results

283080

In [383]:
#Avg number of terms per study 
grpdesc_no_results = average_study("description", "Design_Groups")
grpdesc_no_results

,avg
0,1.823208
